In [1]:
import jinja2, IPython
from coffeetools import coffee
env = jinja2.Environment( variable_start_string='#{', variable_end_string='}')

In [2]:
%%file _config.yml
baseurl: /djatashape

Overwriting _config.yml


In [7]:
render = lambda s: jinja2.Template(s).render(**globals())

In [8]:
js_files = ['/bower_components/pegjs/peg-0.9.0.min.js','/bower_components/codemirror/lib/codemirror.js','/bower_components/d3/d3.min.js']
[js_files.append('/bower_components/codemirror/'+v) for v in ['addon/fold/foldcode.js', 'addon/fold/foldgutter.js']]
css_files = ['/bower_components/bootstrap/dist/css/bootstrap.min.css','/bower_components/bootstrap/dist/css/bootstrap-theme.min.css','/bower_components/codemirror/lib/codemirror.css']

In [9]:
head = IPython.display.HTML(render("""{% for src in js_files %}<script src="http://localhost:4000/djatashape{{src}}"></script>{% endfor %}"""))
head

In [10]:
with open('djatashape.peg.js', 'r') as f:
    parser = f.read()
init_parser = IPython.display.Javascript(coffee.compile(render("""
window.parser = (PEG.buildParser \"\"\"{{parser}}\"\"\").parse
"""),bare=True))

In [11]:
%%html
    <div id="editor"></div>
    <div id="preview"></div>


In [12]:
editor = IPython.display.Javascript(coffee.compile("""
window.cm = CodeMirror d3.select('#editor').node(),
    theme: "blackboard"
    lineNumbers: true
    lineWrapping: true
    readOnly: false
    gutters: ["CodeMirror-linenumbers", "CodeMirror-foldgutter"]
preview = d3.select '#preview'
    .selectAll 'pre > code'
    .data [0]
    .enter().append('pre').append 'code'
cm.on 'change', ()=>
    preview.text JSON.stringify parser(cm.getValue()), null, 2
    
cm.setValue \"\"\"
10 * 2 * {
  day: datetime,
  first_name: ?string,
  last_name: string,
  info: {
      filename: string,
      market: 2 * categorical[values=['fish','trampoline','hot air balloons']]
  }
}
\"\"\"
"""))

In [13]:
index = env.from_string("""---
---
<!doctype html>
<head>
    #{head.data.replace('http://localhost:4000/djatashape','{{site.baseurl}}')}
    {% for src in css_files %}<link rel="stylesheet" type="text/css" href="{{site.baseurl}}#{src}"/>{% endfor %}
</head>
<body>
    <script>#{init_parser.data}</script>
    <div class="container">
        <h1>DataShape in the Browser</h1>
        <p>Type a DataShape Grammar into the editor and it's Javascript serialization is returned below.</p>
        <hr>
        <div class="row"  id="editor"></div>
        <hr>
        <div class="row" id="preview"></div>
    </div>
</body>
<script>#{editor.data}</script>
""").render(**globals())
with open('index.html','w') as f:
    f.write(index)